In [1]:
import requests, bs4, re, os
import numpy as np
import pandas as pd
from datetime import date

pwd = "." #os.path.dirname(os.path.realpath(__file__))

file = "cams.xls"
data = pd.read_excel(pwd+'/'+file, sheet_name="Sheet1")
file1 = "kfin.xls"
data1 = pd.read_excel(pwd+'/'+file1, sheet_name="Consolidated Account Statement")

In [2]:
data.head()

,MF_NAME,PRODUCT_CODE,SCHEME_NAME,Type,TRADE_DATE,TRANSACTION_TYPE,DIVIDEND_RATE,AMOUNT,UNITS,PRICE,BROKER
0,HDFC Mutual Fund,HGFGT,HDFC Balanced Advantage Fund-DIR-Gr-(formerly ...,Balanced,20-APR-2023,Purchase,NaN,100.00,0.285,350.337,Direct
1,HDFC Mutual Fund,HGFGT,HDFC Balanced Advantage Fund-DIR-Gr-(formerly ...,Balanced,21-APR-2023,Registered,NaN,NaN,NaN,NaN,Direct
2,HDFC Mutual Fund,HGFGT,HDFC Balanced Advantage Fund-DIR-Gr-(formerly ...,Balanced,19-JUN-2023,SIP Purchase,NaN,1999.90,5.404,370.074,Direct
3,HDFC Mutual Fund,HGFGT,HDFC Balanced Advantage Fund-DIR-Gr-(formerly ...,Balanced,31-JUL-2023,Address Updated from KRA Data,NaN,NaN,NaN,NaN,Direct
4,HDFC Mutual Fund,HGFGT,HDFC Balanced Advantage Fund-DIR-Gr-(formerly ...,Balanced,17-AUG-2023,Purchase,NaN,899.96,2.287,393.462,Direct


In [3]:
data1.head()

,AMC_NAME,TRXN_DATE,POSTED_DATE,SCHEME_CODE,SHORT_NAME,TRXN_DESC,TRXN_AMOUNT,TRXN_UNITS,PURCH_PRICE,STT_TAX,TAX,TOTAL_TAX,TRXN_MODE,TRXN_SIGN,TRXN_ACCOUNT,STAMP_DUTY,TRXN_CHARGE,ISINCODE
0,NaN,21-JUL-2023,21-JUL-2023,EBD1G,Mirae Asset Emerging Bluechip Fund - Direct Pl...,Systematic Investment (2/12),1999.90,16.851,118.6780,0.0,0.0,0.0,N,+,P,0.10,0.0,INF769K01BI1
1,NaN,24-JUL-2023,24-JUL-2023,MCDGG,Axis Mid Cap Fund - Direct Growth,Systematic Investment (1/12),999.95,11.653,85.8100,0.0,0.0,0.0,N,+,P,0.05,0.0,INF846K01EH3
2,NaN,24-JUL-2023,24-JUL-2023,SCAGG,NIPPON INDIA SMALL CAP FUND - DIRECT GROWTH PL...,Sys. Investment (/),999.95,7.979,125.3154,0.0,0.0,0.0,N,+,P,0.05,0.0,INF204K01K15


In [4]:
def get_RTA_ISIN(rta_code, scheme_name):
    search = rta_code + " " + scheme_name.split("(")[0] + " groww"
    search = search.split(" ")
    search = "+".join(search)

    search = search.replace("&","%26")

    gSearch = requests.get('https://www.google.com/search?q='+search)
    soup = bs4.BeautifulSoup(gSearch.text,"html.parser")
    links = soup.find_all("a")


    for link in links:
        if( "groww.in" in link.get("href") ):
            growwUrl = requests.get("https://www.google.com"+link.get("href"))

            rtaCode = re.findall('"rta_scheme_code":".*",',growwUrl.text)
            rtaCode = rtaCode[0].split(':"')[1].split('",')[0]

            isin = re.findall('"isin":".*",',growwUrl.text)
            isin = isin[0].split(':"')[1].split('",')[0]

            schemeName = re.findall('"scheme_name":".*",',growwUrl.text)
            schemeName = schemeName[0].split(':"')[1].split('",')[0]
            schemeName = schemeName.replace(r"\u0026","&")
            #print(schemeName)
            #print(rtaCode,rta_code)

            if( rta_code in rtaCode):
                return isin,schemeName

In [5]:
mfISIN = {}
mfNAME = {}
toolMF = pd.DataFrame()

In [6]:
for i in range(len(list(data["PRODUCT_CODE"]))):
    if(str(data["UNITS"][i]) != "nan"):
        if(data["PRODUCT_CODE"][i] not in mfISIN.keys()):      
            isin,scheme_name = get_RTA_ISIN(data["PRODUCT_CODE"][i], data["SCHEME_NAME"][i])
            print(data["PRODUCT_CODE"][i],isin)

            mfISIN[data["PRODUCT_CODE"][i]] = isin
            mfNAME[data["PRODUCT_CODE"][i]] = scheme_name

        newRow = pd.DataFrame({'symbol': mfNAME[data["PRODUCT_CODE"][i]], 
                                'isin': mfISIN[data["PRODUCT_CODE"][i]], 
                                'trade_date': data["TRADE_DATE"][i], 
                                'quantity':data["UNITS"][i],
                                'price': data["PRICE"][i],}, index = [0])

        toolMF = pd.concat([toolMF,newRow])

HGFGT INF179K01WA6
P8189 INF109K012M7
K123D INF174K01LT0
PP001ZG INF879O01027
LD103G INF200K01QX4
LD346G INF200K01T51


In [7]:
for i in range(len(list(data1["SCHEME_CODE"]))):
    if(float(data1["TRXN_UNITS"][i]) != 0):
        if(data1["SCHEME_CODE"][i] not in mfISIN.keys()):
            isin,scheme_name = (data1["ISINCODE"][i],data1["SHORT_NAME"][i])
            #isin,scheme_name = get_RTA_ISIN(data1["SCHEME_CODE"][i][:-1], data1["SHORT_NAME"][i])
            
            print(data1["SCHEME_CODE"][i],isin)
            mfISIN[data1["SCHEME_CODE"][i]] = isin
            mfNAME[data1["SCHEME_CODE"][i]] = scheme_name

        newRow = pd.DataFrame({'symbol': mfNAME[data1["SCHEME_CODE"][i]], 
                                'isin': mfISIN[data1["SCHEME_CODE"][i]], 
                                'trade_date': data1["TRXN_DATE"][i], 
                                'quantity':data1["TRXN_UNITS"][i],
                                'price': data1["PURCH_PRICE"][i],}, index = [0])

        toolMF = pd.concat([toolMF,newRow])

EBD1G INF769K01BI1
MCDGG INF846K01EH3
SCAGG INF204K01K15


In [8]:
toolMF.to_csv("../"+"mf_transactions.csv", index=False)

In [9]:
mapDf = pd.DataFrame()
mapDf["CODE"] = mfISIN.keys()
mapDf["ISIN"] = mfISIN.values()
mapDf["Name"] = mfNAME.values()
mapDf.to_csv("./"+"RTA_Code_Map.csv", index=False)